# Image Segmentation with PyTorch

人像留色

zoom 背景更换

图像分类是将一张图片自动分成某一类别，而图像分割是需要将图片中的每一个像素进行分类

图像分割可以分为语义分割与实例分割，两者的区别是语义分割中只需要把每个像素点进行分类就可以了，不需要区分是否来自同一个实例，而实例分割不仅仅需要对像素点进行分类，还需要判断来自哪个实例。

语义分割：对图像中的每个像素进行分类，输出一个与输入图像相同大小的分割图，每个像素的值表示其类别。
实例分割：不仅对每个像素进行分类，还需要区分不同实例，输出一个与输入图像相同大小的分割图，每个像素的值表示其所属的实例类别。

全连接层中的节点数就对应着要将图片分为几类。我们将全连接层的输出送入到 softmax 中，就可以获得每个类别的概率，然后通过概率就可以判断输入图片属于哪一个类别了。

二分类用 Sigmoid + BCELoss
多分类用 Softmax + CrossEntropyLoss

## 转置卷积 这是为了网络结构中的decoder部分准备的

复习一下基本的卷积

![image7.png](mdfiles/image7.png)

转置卷积从逻辑上是卷积的一个逆过程，而不是卷积的逆运算

转置卷积只能还原出一个与输入特征图尺寸相同的特征图。

![image8.png](mdfiles/image8.png)

上面的也可以通过

![image9.png](mdfiles/image9.png)


``` python
class torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=True, dilation=1)
```

in_channels：输入特征图的通道数
out_channels：输出特征图的通道数
kernel_size：卷积核的大小
stride：卷积的步长
padding：输入特征图的零填充
groups：分组卷积
bias：是否使用偏置
dilation：卷积核元素之间的间距

padding 是指要在输入特征图的周围补 **dilation * (kernel_size - 1) - padding** 个零



In [1]:
import torch
import torch.nn as nn
import numpy as np
input_feat = torch.tensor([[[[1, 2], [3, 4]]]], dtype=torch.float32)
input_feat


tensor([[[[1., 2.],
          [3., 4.]]]])

In [2]:
kernels = torch.tensor([[[[1, 0], [1, 1]]]], dtype=torch.float32)
kernels

tensor([[[[1., 0.],
          [1., 1.]]]])

In [7]:
convTrans = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0, bias = False)
convTrans.weight=nn.Parameter(kernels)

In [8]:
result = convTrans(input_feat)
result

tensor([[[[1., 0., 2., 0.],
          [1., 1., 2., 2.],
          [3., 0., 4., 0.],
          [3., 3., 4., 4.]]]], grad_fn=<ConvolutionBackward0>)

![image11.png](mdfiles/image11.png)

这个是区分不同stride下转置卷积的效果

如果stride=1，那么只扩大了1圈
如果stride=2，那么扩大了2圈，中间有空隙

# 损失函数

在分割中，含有像素类别的特征图，我们一般称为 Mask。
这张特征图记录着每个像素的真实分类结果

一张图片有一个预测结果，预测结果与真实值就可以计算出一个 Loss

刚才我们也看到了，图像分割的数据标注还是比较耗时的，具体如何标注一张语义分割所需要的图片，下节课我们再一起通过实践探索。


除此之外业界也有很多比较有权威性且质量很高的公开数据集。最著名的就是 COCO 了，链接如下：https://cocodataset.org/#detection-2016。

小猫分割问题，最终只输出 1 个特征图是否可以?

回答：可以。对于小猫分割问题，我们可以将其视为一个二分类问题，即判断每个像素是否属于小猫。最终输出 1 个特征图，表示每个像素的分类结果（属于小猫或不属于小猫）。